In [93]:
# pip install pyodbc

In [1]:
import pandas as pd
import pyodbc
import os

In [2]:
# Define the parameters for connect to SQL Server database
driver_name = "ODBC Driver 17 for SQL Server"
server_name = 'TIT-20230930FXD\\SQLEXPRESS'
database_name = 'brazilian_ecommerce' # database_name = 'master'
trusted_connection = 'yes'

# Define connection string
conn_str = f'DRIVER={driver_name};SERVER={server_name};DATABASE={database_name};Trusted_Connection={trusted_connection}'

In [3]:
# อ่านข้อมูลจากไฟล์ CSV โดยใช้ pandas
df = pd.read_csv(r'data\\olist_customers_dataset.csv')
df["customer_zip_code_prefix"] = df["customer_zip_code_prefix"].astype(object)
df.dtypes

customer_id                 object
customer_unique_id          object
customer_zip_code_prefix    object
customer_city               object
customer_state              object
dtype: object

In [4]:
# sql_cmd = """
#     CREATE TABLE customers(    
#         customer_id VARCHAR(255) NOT NULL PRIMARY KEY,
#         customer_unique_id VARCHAR(255),
#         customer_zip_code_prefix VARCHAR(255),
#         customer_city VARCHAR(255),
#         customer_state VARCHAR(255)
#     );
# """

# with pyodbc.connect(conn_str) as con:
#     con.execute(sql_cmd)

In [5]:
# กำหนดคำสั่ง SQL เพื่อ truncate ตาราง
truncate_query = '''
    TRUNCATE TABLE customers
'''

# Execute คำสั่ง SQL เพื่อ truncate ตาราง
with pyodbc.connect(conn_str).cursor() as cursor:
    cursor.execute(truncate_query)
    
pyodbc.connect(conn_str).commit()

In [6]:
with pyodbc.connect(conn_str) as con:
    cursor = con.cursor()
    for index, row in df.iterrows():
        insert_query = """
            INSERT INTO customers
            VALUES (?, ?, ?, ?, ?)
        """
        # Execute คำสั่ง SQL เพื่อเพิ่มข้อมูล
        cursor.execute(insert_query, tuple(row))
    con.commit()

In [9]:
with pyodbc.connect(conn_str) as con:
    df_query = pd.read_sql("SELECT * FROM customers", con)
    
df_query.info()

C:\Users\Admin\AppData\Local\Temp\ipykernel_6216\711663833.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_query = pd.read_sql("SELECT * FROM customers", conn)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  object
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: object(5)
memory usage: 3.8+ MB
